In [2]:
import os
import glob
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import scipy

In [ ]:
labelDf0 = pd.read_csv("train_labels.csv")
labelDf0 = labelDf0.set_index('planet_id')
labelDf0

# data

In [3]:
tf.random.set_seed(42)
files = glob.glob(os.path.join('train/', '*/*'))
stars = []
for file in files:
    file_name = file.split('\\')[1]
    stars.append(file_name)
stars = np.unique(stars)

import random
random.seed(42)

def split_star_list(file_list, test_ratio=0.6):
    random.shuffle(file_list)
    split_index = int(len(file_list) * (1 - test_ratio))
    train_files = file_list[:split_index]
    test_files = file_list[split_index:]
    return train_files, test_files

train_stars, test_stars = split_star_list(stars)

labelDf = pd.read_csv("train_labels.csv")
labelDf = labelDf.set_index('planet_id')
meanLabels = np.mean(labelDf.mean())
stdLabels = np.std(labelDf.std())
maxLabels = np.max(labelDf.max())
minLabels = np.min(labelDf.min())

trainLabels = labelDf.loc[[int(star) for star in train_stars]]
meanTrainLabels = np.mean(trainLabels.mean())
stdTrainLabels = np.std(trainLabels.std())
maxTrainLabels = np.max(trainLabels.max())
minTrainLabels = np.min(trainLabels.min())

for col in labelDf.columns:
    labelDf.loc[:,col] = (labelDf[col]) / (maxTrainLabels)

# normalize over time and all samples, so we have a mean and a std dev per wavelength for all samples
def calcMeanAndStdOfTrain(train_stars):
    i = 0
    for star in train_stars:
        file_path = 'train/'+str(star)+'/combined.npz'
        with np.load(file_path) as data:
            x = data['a'][0,:,0:283,:]
            if i ==0:
                mean = np.mean(x,axis=(0))
                sumS = np.sum(x**2,axis=0)
            else:
                mean = mean + np.mean(x, axis=(0))
                sumS += np.sum(x**2,axis=0)
            i=i+1
    meanTrain = mean / i
    stdTrain = np.sqrt(sumS / (i*x.shape[0]) - meanTrain**2)    
    return meanTrain, stdTrain
meanTrain, stdTrain = calcMeanAndStdOfTrain(train_stars)

def normalize_over_train(features, labels):
    features = (features - meanTrain) / (stdTrain + 1e-6)
    return features, labels

# normalize over time per samples, so we have a mean and a std dev per wavelength for all samples
def calcMeanAndStdOfTrainPerStar(x):
    mean = np.mean(x,axis=(0))
    sumS = np.sum(x**2,axis=0)
    stdTrain = np.sqrt(sumS / (x.shape[0]) - mean**2)    
    return mean, stdTrain
def normalize_per_sample(features, labels):
    m,s = calcMeanAndStdOfTrainPerStar(features)
    features = (features) / (s + 1e-6)
    return features, labels




def load_npz(star):
    integer_value = tf.strings.to_number(star, out_type=tf.int64)
    python_int = integer_value.numpy()

    file_path = 'train/'+str(python_int)+'/combined.npz'
    try:
        with np.load(file_path) as data:
            features = data['a'][0,:,0:283,:]
            labels = labelDf.loc[python_int].to_numpy()
            features = np.reshape(features,(-1,25,283,4))
            features = np.mean(features,axis=1)
            #features, labels = normalize_per_sample(features,labels)
            features, labels = normalize_over_train(features,labels)
            return features, labels
    except Exception as e:
        print("Error loading file:", e, python_int)
    

def create_dataset(star_list, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices(star_list)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(star_list))
    def load_and_process(x):
        features, labels = tf.py_function(
            func=load_npz,
            inp=[x],
            Tout=[tf.float64, tf.float32]
        )
        return features, labels

    dataset = dataset.map(load_and_process, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(lambda x, y: (tf.ensure_shape(x,tf.TensorShape([225, 283, 4])), tf.ensure_shape(y, tf.TensorShape([283])))) #5625
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


In [46]:
np.savez('helpers_origiData_meanPred.npz',meanTrain=meanTrain, stdTrain=stdTrain,meanLabels=meanLabels,stdLabels=stdLabels,maxTrainLabels=maxTrainLabels)

In [4]:
tf.random.set_seed(42)
batch_size = 64

train_dataset = create_dataset(train_stars, batch_size, shuffle=True)
test_dataset = create_dataset(test_stars, batch_size, shuffle=False)

# CNN

In [5]:
class Reshape1(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x):
        x = tf.transpose(x, perm=[0,2,1,3])
        #x = tf.reshape(x, [-1, self.timepoints, tf.cast(self.wavelengths * self.representations, tf.int32)])
        return x
    
class Reshape11(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x):
        x = tf.transpose(x, perm=[0,2,1])
        #x = tf.reshape(x, [-1, self.timepoints, tf.cast(self.wavelengths * self.representations, tf.int32)])
        return x

class Reshape2(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x_pred, x_confidence):
        x = tf.concat([x_pred, x_confidence], axis = -1)
        
        return x
    
class Reshape22(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x_pred, x_confidence):
        x_pred = tf.expand_dims(x_pred, axis=-1)
        x_confidence = tf.expand_dims(x_confidence, axis=-1)
        x = tf.concat([x_pred, x_confidence], axis = -1)
        return x
    
class reduce(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x):
        mean = tf.reduce_sum(x,axis=-1)
        mean = tf.expand_dims(mean, axis=-1)
        return mean
class reduce1(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x):
        mean = tf.reduce_sum(x,axis=-1)
        return mean
    
class tile(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x,mean):
        x = tf.concat([x,mean],axis=-1)
        return x
    
class tile2(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x,mean):
        x = tf.concat([x,tf.expand_dims(mean,axis=-1)],axis=-2)
        return x
    
class meanOfWavelengths(tf.keras.layers.Layer):
    def __init__(self, concat=True,**kwargs):
        self.concat=concat
        super().__init__(**kwargs)
    def call(self, x):
        m = tf.expand_dims(tf.reduce_mean(x,axis=-1),axis=-1)
        x = tf.concat([x,m],axis=-1)
        return x if self.concat else m


timepoints = 225
representations = 4
wavelengths = 283
targetWavelengths = 283

def cnnDepthwise():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]
    x = meanOfWavelengths()(x)
    # timepoints (225) * wavelengths (284)
    
    #x = Reshape11()(x)
    dim = timepoints
    for i in range(3):
        # depthwise1d filter -> one filter per channel (=wavelength), depth_multiplier tells us how many filters per channel
        x = tf.keras.layers.DepthwiseConv1D(kernel_size=5,strides=1,padding='same', depth_multiplier=2,activation='relu')(x)
        #x = tf.keras.layers.Conv1D(filters=284, kernel_size=(5), padding='valid')(x)
        x = tf.keras.layers.AveragePooling1D(2)(x)
        x = tf.keras.layers.Dense(284)(x)
    
    x = tf.keras.layers.DepthwiseConv1D(kernel_size=5,strides=1,padding='same', depth_multiplier=2,activation='relu')(x)
    x = tf.keras.layers.Dense(283)(x)
    x = Reshape11()(x)
    x = tf.keras.layers.Dense(1000)(x)
    x = tf.keras.layers.Dense(100)(x)
    
    x_pred = tf.keras.layers.Dense(1, activation='linear')(x)
    x_confidence = tf.keras.layers.Dense(1, activation='linear')(x)
    x = Reshape2()(x_pred, x_confidence)

    model = tf.keras.Model(inp, x)
    return model

model = cnnDepthwise() 
#model = cnnM() 
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 225, 283,  │          0 │ -                 │
│ (InputLayer)        │ 4)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 225, 283)  │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mean_of_wavelengths │ (None, 225, 284)  │          0 │ get_item[0][0]    │
│ (meanOfWavelengths) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv1d    │ (None, 225, 568)  │      3,408 │ mean_of_waveleng… │
│ (DepthwiseConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling1d   │ (None, 112, 568)  │          0 │ depthwise_conv1d… │
│ (AveragePooling1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 112, 284)  │    161,596 │ average_pooling1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv1d_1  │ (None, 112, 568)  │      3,408 │ dense[0][0]       │
│ (DepthwiseConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling1d_1 │ (None, 56, 568)   │          0 │ depthwise_conv1d… │
│ (AveragePooling1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 56, 284)   │    161,596 │ average_pooling1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv1d_2  │ (None, 56, 568)   │      3,408 │ dense_1[0][0]     │
│ (DepthwiseConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling1d_2 │ (None, 28, 568)   │          0 │ depthwise_conv1d… │
│ (AveragePooling1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 28, 284)   │    161,596 │ average_pooling1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv1d_3  │ (None, 28, 568)   │      3,408 │ dense_2[0][0]     │
│ (DepthwiseConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 28, 283)   │    161,027 │ depthwise_conv1d… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape11           │ (None, 283, 28)   │          0 │ dense_3[0][0]     │
│ (Reshape11)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 283, 1000) │     29,000 │ reshape11[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 283, 100)  │    100,100 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 283, 1)    │        101 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 283, 1)    │        101 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape2 (Reshape2) │ (None, 283, 2)    │          0 │ dense_6[0][0],  

 Total params: 788,749 (3.01 MB)

 Trainable params: 788,749 (3.01 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
batch = next(iter(train_dataset))
out = model(batch[0])
test_batch = next(iter(test_dataset))
batch[0].dtype ,batch[1].dtype, out.dtype,batch[0].shape ,batch[1].shape, out.shape

(tf.float64,
 tf.float32,
 tf.float32,
 TensorShape([64, 225, 283, 4]),
 TensorShape([64, 283]),
 TensorShape([64, 283, 2]))

In [7]:
def log_likelihood_maxScaling(y_trueMax, y_pred):
    # stdDev_zScorePred = 1/n * sqrt((y_zScore - y_zScoreMean)^2) = 1/n *sqrt(sum( (y-mean)/std - (y_mean-mean)/std )^2) = 1/n * sqrt(sum( (y-y_mean)/std )^2 )) = 1/std * 1/n * sqrt(sum(y-y_mean)^2) = stdDev / std
    # stdDev_zScorePred = stdDev_pred / std
    # y_pred contains 1. y_zScore 2. log(stdDev_zScore)

    y_true = y_trueMax * maxTrainLabels #std + mean   # y_zScore = (y - mean) / std -> y = y_zScore *std + mean

    y_predMax = y_pred[:, :,0]
    log_sigma = y_pred[:, :,1]  # Log of the standard deviation / we predict log(stdDev_zScore) = log(stdDev / std) = log(stdDev) - log(std) -> log(stdDev) = log(stdDev_zScore) + log(std)

    y_pred0 = y_predMax *maxTrainLabels #* std + mean
    sigma = tf.exp(log_sigma)*maxTrainLabels  # Exponentiate to get variance + scale back from zscore 
    logStdDev = tf.math.log(sigma*sigma)# + tf.math.log(max)

    L_pred = -0.5*(tf.math.log(2*np.pi) + logStdDev + tf.square((y_true - y_pred0) / sigma))
    L_ref = -0.5*(tf.math.log(2*np.pi) +  tf.math.log(stdLabels**4) + tf.square((y_true - meanLabels)/(stdLabels*stdLabels)))   # ( (y_true - mean)/std )^2 = y_trueZScore^2  (y_true = y_trueZScore * std + mean)
    L_ideal = -0.5*(tf.math.log(2*np.pi) + tf.math.log((1e-5)**4)) * tf.ones_like(y_predMax)
    #print(L_pred)
    #print(L_ref)
    #print(L_ideal)
    #print(tf.reduce_sum(L_pred),tf.reduce_sum(L_ideal),tf.reduce_sum(L_ref))
    L = (tf.reduce_sum(L_pred) -tf.reduce_sum(L_ref)) / (tf.reduce_sum(L_ideal) - tf.reduce_sum(L_ref))
    
    return L

def log_likelihood_maxScaling_scipy(y_trueMax, y_pred):
    y_true = y_trueMax * maxTrainLabels #std + mean   # y_zScore = (y - mean) / std -> y = y_zScore *std + mean

    y_predMax = y_pred[:, :,0]
    log_sigma = y_pred[:, :,1]  # Log of the standard deviation / we predict log(stdDev_zScore) = log(stdDev / std) = log(stdDev) - log(std) -> log(stdDev) = log(stdDev_zScore) + log(std)

    y_pred0 = y_predMax *maxTrainLabels #* std + mean
    sigma = tf.exp(log_sigma)*maxTrainLabels  # Exponentiate to get variance + scale back from zscore 

    GLL_pred = np.sum(scipy.stats.norm.logpdf(y_true, loc=y_pred0, scale=sigma))
    GLL_true = np.sum(scipy.stats.norm.logpdf(y_true, loc=y_true, scale=(1e-10) * np.ones_like(y_true)))
    GLL_mean = np.sum(scipy.stats.norm.logpdf(y_true, loc=meanLabels * np.ones_like(y_true), scale=(stdLabels*stdLabels) * np.ones_like(y_true)))

    submit_score = (GLL_pred - GLL_mean)/(GLL_true - GLL_mean)
    #print(GLL_pred, GLL_true, GLL_mean)
    
    return submit_score

log_likelihood_maxScaling(batch[1], out),log_likelihood_maxScaling_scipy(batch[1],out)

(<tf.Tensor: shape=(), dtype=float32, numpy=1.0>, 0.9999999999998191)

In [8]:
def combined_loss_mse(y_trueScaled, y_pred):
    y_predScaled = y_pred[:, :,0]
    lossPred = tf.square(y_trueScaled - y_predScaled)#tf.math.abs(y_true_zScore-y_predZScore)

    logConfidence = tf.math.exp(y_pred[:, :,1]) # logSigma = log(sigma / std)  we predict sigma NOT stdDev!!
    largerThanT = tf.greater(logConfidence, tf.exp(20.0))
    logConfidence = tf.where(largerThanT, y_pred[:,:,1] + tf.exp(20.0), logConfidence)    

    loss_2 = tf.square(lossPred-(logConfidence))


    combinedLoss = tf.reduce_sum(lossPred+loss_2*0.1, axis=-1)
    #combinedLoss = tf.reduce_sum(lossMean, axis=-1)
    return combinedLoss

def mean_mae(y_trueScaled, y_pred):
    y_predMean = tf.expand_dims(tf.reduce_mean(y_pred[:,:,0],axis=-1),axis=-1)
    y_trueMean = tf.expand_dims(tf.reduce_mean(y_trueScaled,axis=-1),axis=-1)
    return tf.abs(y_predMean-y_trueMean)

#loss_mse(batch[1],out)
combined_loss_mse(batch[1],out), mean_mae(batch[1],out)

(<tf.Tensor: shape=(64,), dtype=float32, numpy=
 array([ 40.68333 , 162.26318 ,  35.736195, 164.84949 ,  29.588648,
         31.77409 ,  39.459396,  87.99548 ,  33.062145, 146.09706 ,
         56.535645,  31.32294 ,  36.347675,  29.141367, 132.4765  ,
         29.61394 ,  89.67991 ,  39.12401 ,  35.87329 , 132.97298 ,
         31.743832,  37.123276,  43.7431  ,  29.129421,  32.946198,
        130.27525 ,  42.08177 ,  67.48109 ,  46.082497,  34.130802,
        161.15823 ,  30.483656,  60.23376 ,  28.892202,  43.652443,
        132.42764 ,  91.15231 ,  33.524563,  94.55391 ,  50.91206 ,
         33.7087  ,  51.63279 ,  34.587044,  39.019962,  33.717842,
         32.646435,  32.51447 ,  43.238064, 209.43915 ,  70.86312 ,
         50.009995,  44.85477 ,  42.98866 ,  48.334755,  35.171127,
         98.18271 ,  32.240875,  41.226566, 105.30651 , 177.93816 ,
         32.292652,  37.920948,  38.664062,  36.87942 ], dtype=float32)>,
 <tf.Tensor: shape=(64, 1), dtype=float32, numpy=
 array([[0.2

In [10]:
tf.random.set_seed(42)

lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda step: LR_SCHEDULE[step], verbose=0)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="C:/Users/uic33116/Documents/documents/ariel-data-challenge-2024/training_full_model/model-{epoch:02d}.weights.h5",
    save_weights_only=True,  # Set to False if you want to save the entire model
    save_freq=100 * 5,
    verbose=1
)

#optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model = cnnDepthwise() 
model.compile(loss=combined_loss_mse            
              #,metrics=[log_likelihood_maxScaling]
              ,metrics=[mean_mae]
              , optimizer=optimizer)

history = model.fit(train_dataset, 
                    #batch[0],batch[1], #verbose=2,
                    validation_data=test_dataset,
                    #validation_data=(test_batch[0],test_batch[1]),
                    epochs=800, batch_size=batch_size,
                    callbacks=[checkpoint_callback]
                    #callbacks=[lr_callback]
                    )


Epoch 1/800
5/5 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - loss: 62.2147 - mean_mae: 0.3169 - val_loss: 63.7196 - val_mean_mae: 0.3181
Epoch 2/800
5/5 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - loss: 63.0551 - mean_mae: 0.3174 - val_loss: 63.2057 - val_mean_mae: 0.3150
Epoch 3/800
5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - loss: 62.7387 - mean_mae: 0.3134 - val_loss: 62.6828 - val_mean_mae: 0.3118
Epoch 4/800
5/5 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - loss: 58.6279 - mean_mae: 0.2952 - val_loss: 62.1506 - val_mean_mae: 0.3085
Epoch 5/800
5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - loss: 62.8938 - mean_mae: 0.3123 - val_loss: 61.6027 - val_mean_mae: 0.3051
Epoch 6/800
5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - loss: 60.9547 - mean_mae: 0.3057 - val_loss: 61.0467 - val_mean_mae: 0.3016
Epoch 7/800
5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - loss: 60.6030 - mean_mae: 0.3006 - val_loss: 60.4830 - val_mean_mae: 0.2981
Epoch 8/800
5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - loss: 59.2586 - mean_mae: 0.2948 - val_loss: 59.9056 - val_m

KeyboardInterrupt: 

In [60]:
#model.save('originalData_fullM_866_epochs_blowup59_63.keras')
# Save weights
model.save_weights('800_fullModel_linearMean0078_0017_fullLoss_1_3.weights.h5')

# Load weights
#loaded_weights = model.load_weights('170_epochs_accLoss_reluActivation_23_23.weights.h5')

In [ ]:
loaded_model = tf.keras.models.load_model('120_epochs_accLoss31_30.keras')

In [ ]:
# Assuming 'history' is your model's training history
train_loss = history.history['loss']
test_loss = history.history['val_loss']

epochs = range(1, len(train_loss) + 1)

plt.figure(figsize=(12, 6))
plt.plot(epochs, train_loss, 'b', label='Training loss')
#plt.plot(epochs, test_loss, 'r', label='Test loss')
plt.title('Training and Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# eval

In [ ]:
#pred = model.predict(normData)
outputs = model.predict(batch[0])
pred = outputs[:,:,:]
pred[0:10:,0:2,0]*maxLabels, batch[1][0:10:,0:2]*maxLabels ,np.exp(pred[0:10:,0:2,1])*maxLabels

In [ ]:
np.mean(outputs[:,:,0],axis=-1), np.mean(batch[1],axis=-1)

In [ ]:
outputs[1]

In [ ]:
outputs = model.predict(test_batch[0])
print('overall',(log_likelihood_maxScaling(test_batch[1],outputs)))
#for i in range(batch_size):
#    print(f'batch {i}',(log_likelihood_maxScaling(batch[1][i,:],outputs[i:i+1,:,:])))

In [ ]:
fig = go.Figure()
for i in range(4):#[2,6,10,20,100]:
    fig.add_trace(go.Scatter(y=batch[0][i,:,0,0],mode='markers',name=f'f_{i}',marker=dict(size=3)))

fig.show()

In [ ]:
fig = go.Figure()
for i in range(10): #range(12):# 
    fig.add_trace(go.Scatter(y=batch[1][i,:],mode='markers',name=f'gt_{i}',marker=dict(size=3)))
    fig.add_trace(go.Scatter(y=pred[i,:,0],mode='markers',name=f'pred_{i}',marker=dict(size=3)))
fig.show()

In [ ]:
for i in range(10): #range(12):#
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=batch[1][i,:],mode='markers',name=f'gt_{i}',marker=dict(size=3)))
    fig.add_trace(go.Scatter(y=pred[i,:,0],mode='markers',name=f'pred_{i}',marker=dict(size=3)))
    fig.show()